Собираем все отзывы со страницы товара

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json

Тестовый прогон

In [2]:
link = 'https://pop-music.ru/products/nojnoy-pereklyuchatel-stagg-sswb1-888880011037/'

In [3]:
req = requests.get(link)
soup = BeautifulSoup(req.text, 'html.parser')

In [6]:
reviews = soup.find('div', attrs={'class':'page-section productfull__reviews', 'id':'reviews'})

In [7]:
type(reviews)

bs4.element.Tag

In [11]:
total_rate_class = reviews.find('div', attrs={'class':'productfull__reviews-total-rate'} )

In [13]:
total_rate = total_rate_class.find('strong')
total_rate.text

'4.6'

In [16]:
reviews_list_class = reviews.find('div', attrs={'class':'productfull__reviews-list'} )

In [21]:
reviews_list = []
reviews_list = reviews_list_class.find_all('div', attrs={'class':'js_reviews_list'})

In [23]:
len(reviews_list)

5

In [24]:
test = reviews_list[0]

In [26]:
test_rate_in_stars = len(test.find_all('i', attrs={'class':'rate__star is-full'}))

In [27]:
test_rate_in_stars

5

In [25]:
test.find_all('i', attrs={'class':'rate__star is-full'})

[<i class="rate__star is-full"></i>,
 <i class="rate__star is-full"></i>,
 <i class="rate__star is-full"></i>,
 <i class="rate__star is-full"></i>,
 <i class="rate__star is-full"></i>]

In [29]:
test_text = test.find('div', attrs={'class':'review__text'}).text

In [30]:
test_text

'Здравствуйте, этот переключатель подойдет для BOSS KATANA-50'

In [31]:
test_author = test.find('strong', attrs={'class':'review__name'}).text
test_date = test.find('span', attrs={'class':'review__date'}).text
print(test_author, test_date)

Алексей  30.10.2019


In [34]:
product_name = soup.find('h1', attrs={'class':'productfull__title'}).text
product_id = soup.find('div', attrs={'class':'productfull__code'}).text

In [36]:
product_name = product_name.strip()
product_name

'НОЖНОЙ ПЕРЕКЛЮЧАТЕЛЬ STAGG SSWB1'

In [37]:
product_id = product_id.replace('Артикул: ', '').strip()
product_id

'888880011037'

In [42]:
review_attrs_list = []

In [43]:
for review_html in reviews_list:
    review_attrs = {}
    rate_in_stars = len(review_html.find_all('i', attrs={'class':'rate__star is-full'}))
    text = review_html.find('div', attrs={'class':'review__text'}).text 
    author = review_html.find('strong', attrs={'class':'review__name'}).text
    date = review_html.find('span', attrs={'class':'review__date'}).text
    
    review_attrs['product_name'] = product_name
    review_attrs['product_id'] = product_id
    review_attrs['rate'] = int(rate_in_stars)
    review_attrs['text'] = text
    review_attrs['author'] = author
    review_attrs['date'] = date
    
    review_attrs_list.append(review_attrs)
    

In [7]:
from pprint import pprint

In [45]:
pprint(review_attrs_list)

[{'author': 'Алексей ',
  'date': '30.10.2019',
  'product_id': '888880011037',
  'product_name': 'НОЖНОЙ ПЕРЕКЛЮЧАТЕЛЬ STAGG SSWB1',
  'rate': 5,
  'text': 'Здравствуйте, этот переключатель подойдет для BOSS KATANA-50'},
 {'author': 'Artem',
  'date': '02.08.2019',
  'product_id': '888880011037',
  'product_name': 'НОЖНОЙ ПЕРЕКЛЮЧАТЕЛЬ STAGG SSWB1',
  'rate': 5,
  'text': 'Добрый день!\n\nЭтот переключатель подойдет для Roland Cube 30?'},
 {'author': 'Сергей',
  'date': '17.04.2019',
  'product_id': '888880011037',
  'product_name': 'НОЖНОЙ ПЕРЕКЛЮЧАТЕЛЬ STAGG SSWB1',
  'rate': 5,
  'text': 'Добрый день, а можно его использовать как обходной переключатель '
          'bypass, на сабе krk 10s2?'},
 {'author': 'Алексей ',
  'date': '10.06.2018',
  'product_id': '888880011037',
  'product_name': 'НОЖНОЙ ПЕРЕКЛЮЧАТЕЛЬ STAGG SSWB1',
  'rate': 5,
  'text': 'Здравствуйте. А совместим ли этот фут-свитч с комбиком Feder '
          'Mustang II? \n'
          'Оригинального фут-свитча by Fender

In [2]:
import pandas as pd

In [47]:
df_rate = pd.DataFrame(review_attrs_list)

In [48]:
df_rate

,product_name,product_id,rate,text,author,date
0,НОЖНОЙ ПЕРЕКЛЮЧАТЕЛЬ STAGG SSWB1,888880011037,5,"Здравствуйте, этот переключатель подойдет для ...",Алексей,30.10.2019
1,НОЖНОЙ ПЕРЕКЛЮЧАТЕЛЬ STAGG SSWB1,888880011037,5,Добрый день!\n\nЭтот переключатель подойдет дл...,Artem,02.08.2019
2,НОЖНОЙ ПЕРЕКЛЮЧАТЕЛЬ STAGG SSWB1,888880011037,5,"Добрый день, а можно его использовать как обхо...",Сергей,17.04.2019
3,НОЖНОЙ ПЕРЕКЛЮЧАТЕЛЬ STAGG SSWB1,888880011037,5,Здравствуйте. А совместим ли этот фут-свитч с ...,Алексей,10.06.2018
4,НОЖНОЙ ПЕРЕКЛЮЧАТЕЛЬ STAGG SSWB1,888880011037,3,"Из плюсов: удобная для нажатия кнопка, длинный...",Лудина Ольга,01.01.2012


Собираем в одну функцию  

In [1]:
def get_product_reviews(link):
    
    req = requests.get(link)
    soup = BeautifulSoup(req.text, 'html.parser')

    review_attrs_list = []
    
    product_name = soup.find('h1', attrs={'class':'productfull__title'}).text
    product_name = product_name.strip()
    product_id = soup.find('div', attrs={'class':'productfull__code'}).text
    product_id = product_id.replace('Артикул: ', '').strip()
    
    reviews = soup.find('div', attrs={'class':'page-section productfull__reviews', 'id':'reviews'})
    reviews_list_class = reviews.find('div', attrs={'class':'productfull__reviews-list'} )
    if reviews_list_class:
        reviews_list = reviews_list_class.find_all('div', attrs={'class':'js_reviews_list'})

        for review_html in reviews_list:
            review_attrs = {}
            rate_in_stars = len(review_html.find_all('i', attrs={'class':'rate__star is-full'}))
            text = review_html.find('div', attrs={'class':'review__text'}).text 
            author = review_html.find('strong', attrs={'class':'review__name'}).text
            date = review_html.find('span', attrs={'class':'review__date'}).text

            review_attrs['product_name'] = product_name
            review_attrs['product_id'] = product_id
            review_attrs['rate'] = int(rate_in_stars)
            review_attrs['text'] = text
            review_attrs['author'] = author
            review_attrs['date'] = date

            review_attrs_list.append(review_attrs)
        
    return review_attrs_list 
    

In [3]:
test_link = 'https://pop-music.ru/products/gitara-akusticheskaya-yamaha-f-310-nat-f310-888880017135/'


In [6]:
len(get_product_reviews(test_link))

57

In [8]:
pprint(get_product_reviews(test_link)[0])

{'author': 'Кирилл',
 'date': '07.11.2021',
 'product_id': '888880017135',
 'product_name': 'ГИТАРА АКУСТИЧЕСКАЯ YAMAHA F-310 NAT (F310)',
 'rate': 5,
 'text': 'Здравствуйте, хотел приобрести эту гитару. Сколько будет стоить '
         'доставка до Республики Саха Якутия г. Алдан'}


Ссылки собраны заранее, соберем данные по каждой из них

In [13]:
all_links = []

with open('product_links.txt', 'r') as f:
    for line in f:
        all_links.append(line.strip())
    
len(all_links)

11717

In [14]:
all_links[:10]

['https://pop-music.ru/products/audiointerfeys-focusrite-clarett-4pre-888880029674/',
 'https://pop-music.ru/products/klassicheskaya-gitara-stagg-25th-scl-maho-888880028463/',
 'https://pop-music.ru/products/perehodnik-soundking-cxa004-888880021470/',
 'https://pop-music.ru/products/plastik-evans-bd20gb4s-888880020015/',
 'https://pop-music.ru/products/gitarnyy-kabel-soundking-bjj031-15ft/',
 'https://pop-music.ru/products/bas-gitara-schecter-omen-5-blk-l-h-888880006652/',
 'https://pop-music.ru/products/plastik-evans-b16uv1-888880025353/',
 'https://pop-music.ru/products/akusticheskaya-gitara-stagg-sa20d-red-888880022374/',
 'https://pop-music.ru/products/predusilitel-art-usbdualpre-888880000341/',
 'https://pop-music.ru/products/top-lok-schaller-r1-art-13120701-888880011641/']

In [26]:
df_full = pd.DataFrame(columns=colunn_names)
df_full

,author,date,product_id,product_name,rate,text


In [27]:
from tqdm import tqdm 

In [34]:
wrong_links = []
for link in tqdm(all_links):
    reviews = []
    try:
        reviews = get_product_reviews(link)
    except:
        wrong_links.append(link)
        
    if reviews:
        df_full = pd.concat([df_full, pd.DataFrame(reviews)])
    

100%|██████████████████████████████████████████████████████████████████████████| 11717/11717 [5:10:07<00:00,  1.59s/it]


In [36]:
df_full.head()

,author,date,product_id,product_name,rate,text
0,Егор Гумеров,15.10.2017,888880022374,АКУСТИЧЕСКАЯ ГИТАРА STAGG SA20D RED,4,Это моя первая гитара. Консультант ответил на ...
0,Гость,15.06.2010,888880000341,ПРЕДУСИЛИТЕЛЬ ART USBDUALPRE,3,Пожалуйста скажите кто-нибудь у кого он есть т...
1,Гость,03.06.2010,888880000341,ПРЕДУСИЛИТЕЛЬ ART USBDUALPRE,5,клевая вещь. ка будет в МСК?
2,Гость,18.05.2010,888880000341,ПРЕДУСИЛИТЕЛЬ ART USBDUALPRE,3,Пришлось снизить оценку -1 т.к. выснилось что ...
3,Гость,18.05.2010,888880000341,ПРЕДУСИЛИТЕЛЬ ART USBDUALPRE,4,"Под ХР сразу не заработал, дрова надо везти с ..."


In [37]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11214 entries, 0 to 7
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   author        11214 non-null  object
 1   date          11214 non-null  object
 2   product_id    11214 non-null  object
 3   product_name  11214 non-null  object
 4   rate          11214 non-null  object
 5   text          11214 non-null  object
dtypes: object(6)
memory usage: 613.3+ KB


In [38]:
len(wrong_links)

4

In [39]:
wrong_links

['https://pop-music.ru/products/barabannyie-palochki-stagg-sm5an-888880010522/',
 'https://pop-music.ru/products/gitarnyy-preamp-yerasov-fetube-uber-higain-uh-5-888880023455/',
 'https://pop-music.ru/products/kov%D1%91r-roland-tdm-20-888880001275/',
 'https://pop-music.ru/products/mikrofonnyiy-kabel-planet-waves-pw-cmic-10-888880006165/']

In [43]:
df_full = df_full.reset_index()

In [46]:
df_full.drop('index', axis=1, inplace=True)

In [67]:
df_full.sample(20)

,author,date,product_id,product_name,rate,text
10635,Игорь Скоробогатько,22.12.2018,888880014669,Подвес для микрофона STAGG SHOMOH,5,Подойдет ли к этой стойке? https://pop-music.r...
131,Бондарь Евгений,29.03.2010,88880000874,КОМПЛЕКТ МИКРОФОНОВ BEHRINGER C-2,3,Я бы наказал человека принёсшего в студию тако...
6673,Мангушев Дмитрий,24.03.2010,888880004632,БЛОК ПИТАНИЯ ROBITON SN500S,5,Хороший блок.. универсальный... я выкинул из н...
100,Григорий,20.02.2018,888880011038,НОЖНОЙ ПЕРЕКЛЮЧАТЕЛЬ STAGG SSWB2,5,Для комбика BOSS KATANA-50 подойдет?
590,Алексей,12.04.2021,888880024817,Электроакустическая гитара BATON ROUGE X2S/ACE...,5,А где разъём для подключения кабеля в ней ??
7130,Екатерина,14.06.2020,888880025144,Поп-Фильтр FZONE PF-8,4,"Подскажите пожалуйста, подойдёт ли это поп фил..."
541,Вадим Аммосов,09.03.2018,888880000355,МИКРОФОН AKG PERCEPTION 120 (P120),5,"по сравнению с AT2020, что лучше ? \n"
10051,Сафронов Дмитрий,03.03.2016,3620000001,ЛАМПОВЫЙ ПРЕДУСИЛИТЕЛЬ ART TUBE MP,5,"Скажите пожалуйста, как он будет звучать на за..."
1676,Петр Султангалеев,18.01.2018,888880021107,МИКШЕРНЫЙ ПУЛЬТ SOUNDCRAFT Signature 10,5,Подскажите пожалуйста. В него можно сразу писа...
2228,RADIY,12.09.2017,888880019089,Комбик FENDER ACOUSTASONIC 15 COMBO,5,"Добрый день. Все оборудование , представленное..."


In [48]:
df_full.to_csv('reviews.csv')